## Create a Direct Lake version of your semantic model and clone/rebind a report to it
### Useful if you want to:
- Take Direct Lake Mode for a spin - see how your model & report will perform before investing time in new ETL
- Create a Direct Lake replica of your model & report (typically use QSO for high concurrency, but this is an option too)

Note - OneLake Integration must be enabled for your semantic model. Also use a Lakehouse **without** schema.

In [ ]:
%pip install semantic-link-labs

In [ ]:
# Set all the names
ds_name = 'Flights43M'
new_ds_name = ds_name + '_OneLakeIntegration'
reportname = 'Flights43M'
new_reportname = reportname + '_OneLakeIntegration'
lakehouse_name = 'TryoutLH'

In [ ]:
# Before you run this cell, OneLake Integration needs to be enabled for the semantic model
import sempy_labs as labs
labs.export_model_to_onelake(dataset=ds_name, destination_lakehouse=lakehouse_name)

In [ ]:
# Before running this cell, make sure the desired LH is attached
import sempy.fabric as fabric
import sempy_labs as labs
from sempy_labs import directlake as dl 
from sempy_labs import migration as mgrt

# lh_tables = ['Airports', 'Carriers', 'ColumnsFP', 'Date', 'FieldPicker', 'Flights', 'MeasuresFP', 'MeasureTable', 'Planes', 'Time']
# lh_tables = ['Airports', 'Carriers', 'Date', 'Flights', 'MeasureTable', 'Planes', 'Time'] #list all tables that are not Field Parameters or calc groups
partitions = fabric.list_partitions(ds_name)
nonFPtablesList = partitions[~(partitions['Query'].str.contains('NAMEOF', na=False)) & ~(partitions['Source Type']=='CalculationGroup')]
lh_tables = nonFPtablesList['Table Name'].unique().tolist()

# Create DL model based on LH
dl.generate_direct_lake_semantic_model(dataset = new_ds_name, lakehouse = lakehouse_name, lakehouse_tables = lh_tables)

# Migrate model objects to new model
mgrt.migrate_model_objects_to_semantic_model(ds_name, new_ds_name)
mgrt.migrate_field_parameters(ds_name, new_ds_name)

# Refresh the model
fabric.refresh_dataset(dataset=new_ds_name)

In [ ]:
# Validate Migration
validation_df = labs.migration.migration_validation(ds_name, new_ds_name)
print(validation_df.to_string())

In [ ]:
# Clone the report
from sempy_labs import report as rpt
rpt.clone_report(reportname, new_reportname)

# Bind new report to new model
rpt.report_rebind(new_reportname, new_ds_name)

## Permanently move to a Direct Lake model and clone/bind report to it
### Useful if you want to:
- take advantage of Direct Lake mode benefits (separate refresh and query, lower peak memory, reduce data duplication with table shortcuts, etc.)
- leverage other Fabric ETL options (pipeline, notebook, Dataflow Gen 2 instead of traditional import Power Query, Dataflow Gen 1)

In [ ]:
# This cell is to create a model based on replacement OneLake tables, migrate model objects, and then clone/rebind the report
import sempy.fabric as fabric
import sempy_labs as labs
from sempy_labs import directlake as dl 
from sempy_labs import migration as mgrt
from sempy_labs import report as rpt

# Set all the names
ds_name = 'Flights43M'
new_ds_name = ds_name + '_NewTables'
reportname = 'Flights43M'
new_reportname = reportname + '_NewTables'
lakehouse_name = 'New_Tables_LH'

# Get the list of tables from the current model (that are not Field Parameters or Calculation Groups)
partitions = fabric.list_partitions(ds_name)
nonFPtablesList = partitions[~(partitions['Query'].str.contains('NAMEOF', na=False)) & ~(partitions['Source Type']=='CalculationGroup')]
lh_tables = nonFPtablesList['Table Name'].unique().tolist()

# Create DL model based on LH
dl.generate_direct_lake_semantic_model(dataset = new_ds_name, lakehouse = lakehouse_name, lakehouse_tables = lh_tables)

# Migrate model objects to new model
mgrt.migrate_model_objects_to_semantic_model(ds_name, new_ds_name)
mgrt.migrate_field_parameters(ds_name, new_ds_name)

# Refresh the model
fabric.refresh_dataset(dataset=new_ds_name)

# Clone the report
rpt.clone_report(reportname, new_reportname)

# Bind new report to new model
rpt.report_rebind(new_reportname, new_ds_name)

# Validate Migration
# validation_df = labs.migration.migration_validation(ds_name, new_ds_name)
# print(validation_df.to_string())